# Ideas for TerraRef Consulting Project

In [15]:
install.packages(c("knitr","lubridate", "plyr","data.table"), repos='http://cran.us.r-project.org')


The downloaded source packages are in
	‘/private/var/folders/44/z41l8sf111x6k2bjrjbdjhl80000gn/T/RtmpX0OOGB/downloaded_packages’


In [17]:
setwd(dir = '../')


In [26]:
install.packages("dplyr",repos='http://cran.us.r-project.org')

also installing the dependencies ‘assertthat’, ‘R6’, ‘lazyeval’, ‘DBI’, ‘BH’




The downloaded source packages are in
	‘/private/var/folders/44/z41l8sf111x6k2bjrjbdjhl80000gn/T/RtmpX0OOGB/downloaded_packages’


In [32]:

load("samples.Rdata")
library(knitr)

phenotypes <- cbind(genotype = 9914+1:500, 
                    ensemble.samples$Miscanthus_x_giganteus)

summary(phenotypes)



Warning message:
In readChar(con, 5L, useBytes = TRUE): cannot open compressed file 'samples.Rdata', probable reason 'No such file or directory'

ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


    genotype         Vcmax          c2n_leaf      cuticular_cond   
 Min.   : 9915   Min.   :22.43   Min.   : 3.757   Min.   :  303.2  
 1st Qu.:10040   1st Qu.:27.99   1st Qu.:20.562   1st Qu.: 2413.0  
 Median :10164   Median :29.64   Median :29.395   Median : 4462.3  
 Mean   :10164   Mean   :29.64   Mean   :31.898   Mean   : 6553.0  
 3rd Qu.:10289   3rd Qu.:31.25   3rd Qu.:40.656   3rd Qu.: 8031.1  
 Max.   :10414   Max.   :36.73   Max.   :91.917   Max.   :53419.6  
      SLA         quantum_efficiency leaf_respiration_rate_m2 stomatal_slope.BB
 Min.   : 9.855   Min.   :0.05248    Min.   :0.7624           Min.   :2.581    
 1st Qu.:14.925   1st Qu.:0.05960    1st Qu.:1.7851           1st Qu.:2.950    
 Median :15.243   Median :0.06140    Median :2.0168           Median :3.000    
 Mean   :15.162   Mean   :0.06139    Mean   :2.0165           Mean   :2.999    
 3rd Qu.:15.545   3rd Qu.:0.06327    3rd Qu.:2.2505           3rd Qu.:3.051    
 Max.   :18.694   Max.   :0.06841    Max.   

In [19]:
library(data.table)
library(lubridate)
setwd("out/")
files <- data.table(run = 9915:10141)#10482:11048)
allout <- list()
for(i in 1:length(files$run)){
  run <- files$run[i]
  
  filename = file.path(run, "biocro_output.RData")
  if(file.exists(filename)){
    load(filename)
    # Palmer Drought Severity Index for Ohio valley
    # http://www.ncdc.noaa.gov/extremes/cei/graph/ce/3/06-08
    # 1997 wet
    # 1999 dry
    # 2000 normal
    # 2004 wet
    # 2007 dry
    if(!is.null(biocro_result)){
      .tmp <- cbind(run = run, biocro_result[year %in% c(1997, 1999, 2000, 2004, 2007) & 
        (lat %in% c(42.625, 42.375, 40.625, 40.375, 37.625, 37.875) & 
         lon %in% c(-88.625, -88.375)), 
         list(plot = lat*10000-lon*100,
         site = ifelse(lat > 42, 'north', ifelse(lat > 40, 'central', 'south')),
         year, date = ymd(paste(year, 01, 01)) + days(doy), Stem, Leaf, Root, Rhizome, tmax, tmin, tavg, precip)])
      allout[[i]]  <- cbind(.tmp[,list(sitename = site, plotid = LETTERS[frank(plot, ties.method = 'dense')]), by = 'site'], .tmp)
      rm(biocro_result, .tmp)
    }
    
  }
}
all <- rbindlist(allout)
save(all, file = "sorghum_simulation.RData")



In [31]:
load("sorghum_simulation.RData")
library(data.table)
library(lubridate)
all$plot <- NULL
all$site <- NULL
set.seed(100)

all[, `:=`(LAI = Leaf * (0.66 + runif(1:length(Leaf), -0.01 * Leaf / 10, 0.01*Leaf/10)))]
all[, `:=`(NDVI = 0.9 + (-0.66 * 0.76 * exp(-1.1 * LAI + rnorm(1:length(LAI), 0, LAI/100))))]
## height proportional to biomass via volume 
all[, `:=` ( Height = (pi / 3 )*(rnorm(1:length(Stem), Stem/100, Stem/1000) + runif(1:length(Stem), Stem / 1000, 2*Stem / 1000) + Stem * (Stem / 50)^2)^(1/3)-0.02)  ]

#save(all, file = "sorghum_simulation.RData")
#load('sorghum_simulation.RData')

library(dplyr)

errors <- function(x) {
  e1 <- runif(1:length(x), x/1000, x/800)
  e2 <- rnorm(1:length(x), 2*log(1+x), sd(x) * x/100)
  e3 <- min(e1, e2) + e1 / 1000 + e2 / 1000
  e <- e1 * runif(1:length(x)) + e2 * rlnorm(1:length(x), -0.5, 0.1) + e3 * rgamma(1:length(x), 2, 4)
  big <- x > 0.01
  x[big] <- x[big] + e[big]
  x[x < 0.01] <- 0
  x <- round(x, 2)
  return(x)
}

errors <- compiler::cmpfun(errors)
sorghum_simulation <- tbl_df(all)  %>% 
  mutate(genotype = run, Tmax = round(tmax,1), Tmin = round(tmin, 1), Tavg = round(tavg, 1), 
         precipitation = round(precip, 2), 
         Stem = errors(Stem), Leaf = errors(Leaf), Root = errors(Root), LAI = errors(LAI), 
         NDVI = errors(NDVI), Height = errors(Height)) %>% 
  select(-run)

sorghum_simulation <- sorghum_simulation %>%
  mutate(year = 2020 + frank(year, ties.method = 'dense')) %>% 
  dplyr::select(-tmax, -tmin, -tavg)

year(sorghum_simulation$date) <- sorghum_simulation$year
write.csv(phenotypes, 'phenotypes.csv')

s <- sorghum_simulation %>% setDT
sorghum_observations <- s[,list(sitename, plotid, genotype, date, Stem, Leaf, Root, Rhizome, precip, LAI, NDVI, Height)] 

write.csv(sorghum_observations, 'observations.csv')

sorghum_met <- unique(s[,list(sitename, plotid, date, Tmax, Tmin, Tavg, precipitation)])

write.csv(sorghum_met, 'met.csv')
#normalized_difference_vegetation_index
#leaf area index
variables <- fread("~/Downloads/BETYdb Variables - Sheet 1.csv")
knitr::kable(variables[Name %in% c(colnames(phenotypes), colnames(sorghum_simulation)), list(variable_id = id, name = Name, standard_name, units = Units, Description)])



ERROR: Error in `[<-.data.table`(x, j = name, value = value): Cannot use := to add columns to a null data.table (no columns), currently. You can use := to add (empty) columns to a 0-row data.table (1 or more empty columns), though.


ERROR: Error in `[<-.data.table`(x, j = name, value = value): Cannot use := to add columns to a null data.table (no columns), currently. You can use := to add (empty) columns to a 0-row data.table (1 or more empty columns), though.


ERROR: Error in eval(expr, envir, enclos): object 'Leaf' not found


ERROR: Error in eval(expr, envir, enclos): object 'LAI' not found


ERROR: Error in rnorm(1:length(Stem), Stem/100, Stem/1000): object 'Stem' not found


...

ERROR: Error in is.data.frame(x): object 'sorghum_observations' not found


ERROR: Error in unique(s[, list(sitename, plotid, date, Tmax, Tmin, Tavg, precipitation)]): object 's' not found


ERROR: Error in is.data.frame(x): object 'sorghum_met' not found


ERROR: Error in fread("~/Downloads/BETYdb Variables - Sheet 1.csv"): File is empty: /var/folders/44/z41l8sf111x6k2bjrjbdjhl80000gn/T//RtmpX0OOGB/file6c9c71022f09


ERROR: Error in inherits(x, "list"): object 'variables' not found


In [29]:
install.packages(c("ggthemes", "GGally"), repos='http://cran.us.r-project.org')

also installing the dependency ‘reshape’




The downloaded source packages are in
	‘/private/var/folders/44/z41l8sf111x6k2bjrjbdjhl80000gn/T/RtmpX0OOGB/downloaded_packages’


In [30]:
### ---- end part
library(data.table)
library(dplyr)

phenotypes <- fread("phenotypes.csv")
sorghum_simulation <- fread("sorghum_simulation.csv")
library(ggplot2)
library(ggthemes)
library(GGally)
theme_set(theme_tufte())

p <- phenotypes %>% dplyr::select(Vcmax, c2n_leaf, cuticular_cond, SLA, quantum_efficiency,
                                  leaf_respiration_rate_m2, stomatal_slope.BB, growth_respiration_coefficient,
                                  Jmax, chi_leaf, extinction_coefficient_diffuse) %>% setDT

ggpairs(p,  
        lower = list(continuous = 'density'),
        upper = list(continuous = 'blank'),
        axisLabels = 'internal')
s <- sorghum_simulation %>% 
  mutate(Day = yday(date), 
         line = as.factor(paste('Sim', genotype)),
         Biomass = Stem + Leaf) %>% setDT

## figure out best / worst
s[year == 2023][which.max(Stem)]
s[year == 2023 & month(date) == 10][which.min(Stem)]
theme_set(theme_tufte()) 
ggplot() + 
  geom_smooth(data = s[genotype >9950 & genotype < 10000],
              aes(x = Day, y = Biomass, fill = NULL, group = genotype), alpha = 0.1, color = 'grey') +
  geom_smooth(data = s[genotype == 10020],
              aes(x = Day, y = Biomass), color = 'darkgreen') +
  geom_smooth(data = s[genotype == 9934],
              aes(x = Day, y = Biomass), color = 'red') +
  facet_wrap(~year, nrow = 1) 

ERROR: Error in fread("sorghum_simulation.csv"): File 'sorghum_simulation.csv' does not exist. Include one or more spaces to consider the input a system command.



Attaching package: ‘GGally’

The following object is masked from ‘package:dplyr’:

    nasa

Warning message:
In phenotypes %>% dplyr::select(Vcmax, c2n_leaf, cuticular_cond, : restarting interrupted promise evaluationWarning message:
In phenotypes %>% dplyr::select(Vcmax, c2n_leaf, cuticular_cond, : internal error -3 in R_decompress1

ERROR: Error in phenotypes %>% dplyr::select(Vcmax, c2n_leaf, cuticular_cond, : lazy-load database '/anaconda/lib/R/library/magrittr/R/magrittr.rdb' is corrupt


ERROR: Error in identical(class(data), "data.frame"): object 'p' not found


Warning message:
In sorghum_simulation %>% mutate(Day = yday(date), line = as.factor(paste("Sim", : restarting interrupted promise evaluationWarning message:
In sorghum_simulation %>% mutate(Day = yday(date), line = as.factor(paste("Sim", : internal error -3 in R_decompress1

ERROR: Error in sorghum_simulation %>% mutate(Day = yday(date), line = as.factor(paste("Sim", : lazy-load database '/anaconda/lib/R/library/magrittr/R/magrittr.rdb' is corrupt


ERROR: Error in eval(expr, envir, enclos): object 's' not found


ERROR: Error in eval(expr, envir, enclos): object 's' not found


ERROR: Error in fortify(data): object 's' not found


In [25]:
list.files()

[1] "phenotypes.csv"           "sorghum_simulation.RData"

Warning message:
In sorghum_simulation %>% mutate(Day = yday(date), line = as.factor(paste("Sim", : restarting interrupted promise evaluationWarning message:
In sorghum_simulation %>% mutate(Day = yday(date), line = as.factor(paste("Sim", : internal error -3 in R_decompress1

ERROR: Error in sorghum_simulation %>% mutate(Day = yday(date), line = as.factor(paste("Sim", : lazy-load database '/anaconda/lib/R/library/magrittr/R/magrittr.rdb' is corrupt


ERROR: Error in eval(expr, envir, enclos): object 's' not found


ERROR: Error in eval(expr, envir, enclos): object 's' not found


ERROR: Error in as.vector(y): could not find function "theme_tufte"


ERROR: Error in fortify(data): object 's' not found
